In [1]:
import pandas as pd
from collections import defaultdict

# tasks = ['nsp_choice', 'ordering', 'rst', 'segment', 'dissent', 'cloze']
tasks = ['dissent']
#models = ['bert', 'roberta', 'albert', 'gpt2', 'electra', 'bart', \
#         'bert-avg', 'albert-avg', 'bert-large', 'bert-zh', 'bert-es', 'bert-de']
models = ['t5-base']

files = '.'

def read_csv(fname):
    layers = []; dev = []; test = []
    for line in open(fname).readlines()[1:]:
        data = line.split(', ')
        layers.append(int(data[0]))
        dev.append(float(data[1]))
        test.append(float(data[2]))
    df = pd.DataFrame()
    df['layers']=layers
    df['dev']=dev
    df['test']=test
    return df

In [2]:
outputs = defaultdict(list)

for output in ['output1', 'output2', 'output3']:
    for task in tasks:
        if task == 'rst':
            for model in models:
                fname1 = f'{files}/{task}/{output}/{model}_nuclearity.csv'
                data1 = read_csv(fname1)
                for idx, row in data1.iterrows():
                    key = (task+'_nuclearity',model,row['layers'])
                    outputs[key].append(row['test'])
                
                fname2 = f'{files}/{task}/{output}/{model}_relation.csv'
                data2 = read_csv(fname2)
                for idx, row in data2.iterrows():
                    key = (task+'_relation',model,row['layers'])
                    outputs[key].append(row['test'])
                
        else:
            for model in models:
                fname = f'{files}/{task}/{output}/{model}.csv'
                try:
                    data = read_csv(fname)
                except:
                    print(fname)
                    continue
                for idx, row in data.iterrows():
                    key = (task,model,row['layers'])
                    outputs[key].append(row['test'])

In [3]:
mean_std = {}
import numpy as np
for key in outputs.keys():
    mean = np.mean(outputs[key])
    std = np.std(outputs[key])
    mean_std[key] = [mean, std]
print(mean_std)

{('dissent', 't5-base', 1.0): [0.432, 0.00637704215656967], ('dissent', 't5-base', 2.0): [0.4663333333333333, 0.0038586123009300612], ('dissent', 't5-base', 3.0): [0.48166666666666663, 0.004784233364802446], ('dissent', 't5-base', 4.0): [0.5046666666666667, 0.0016996731711975963], ('dissent', 't5-base', 5.0): [0.511, 0.004242640687119289], ('dissent', 't5-base', 6.0): [0.5226666666666667, 0.002624669291337273]}


In [4]:
unique = []
for key in mean_std.keys():
    unique.append(key[0])
updated_tasks = list(set(unique))

In [5]:
# key2col = {
#     'nsp_choice': 'NSP with 4 MC',
#     'ordering': 'Sent. Ordering',
#     'dissent': 'Discourse Connective',
#     'rst_nuclearity': 'Nuclearity',
#     'rst_relation': 'Relation',
#     'segment': 'EDU segmentation',
#     'cloze': 'Cloze'
# }
key2col = {
    'dissent': 'Discourse Connective'
}

In [6]:
!mkdir 'RESULT/MEAN'
!mkdir 'RESULT/STD'

mkdir: RESULT/MEAN: File exists
mkdir: RESULT/STD: File exists


In [7]:
def to_file(fname, data):
    df = pd.DataFrame()
    df['Layer'] = data['Layer']
    #print(data)
    for key in key2col.values():
        if data[key]!= []:
            print(df)
            df[key] = data[key]
        else:
            df[key] = [None] * len(data['Layer'])
    df.to_csv(fname, index=False)

for model in models:
    data_mean = defaultdict(list)
    data_std =  defaultdict(list)
    for task in updated_tasks:
        if 'large' in model:
            data_mean['Layer'] = list(range(1, 25))
            data_std['Layer'] = list(range(1, 25))
            for i in range(1,25):
                key = (task, model, i)
                data_mean[key2col[task]].append(mean_std[key][0]) 
                data_std[key2col[task]].append(mean_std[key][1]) 
        else:
            # data_mean['Layer'] = list(range(1, 13))
            # data_std['Layer'] = list(range(1, 13))
            data_mean['Layer'] = list(range(1, 7)) # just test the first six layers of the encoder
            data_std['Layer'] = list(range(1, 7))
            for i in range(1,13):
                try:
                    key = (task, model, i)
                    print(key)
                    print(mean_std[key][0])
                    data_mean[key2col[task]].append(mean_std[key][0]) 
                    data_std[key2col[task]].append(mean_std[key][1]) 
                except:
                    continue
                    #print(key)
    print(data_mean)
    to_file(f'RESULT/MEAN/{model}.csv', data_mean)
    to_file(f'RESULT/STD/{model}.csv', data_std)

('dissent', 't5-base', 1)
0.432
('dissent', 't5-base', 2)
0.4663333333333333
('dissent', 't5-base', 3)
0.48166666666666663
('dissent', 't5-base', 4)
0.5046666666666667
('dissent', 't5-base', 5)
0.511
('dissent', 't5-base', 6)
0.5226666666666667
('dissent', 't5-base', 7)
('dissent', 't5-base', 8)
('dissent', 't5-base', 9)
('dissent', 't5-base', 10)
('dissent', 't5-base', 11)
('dissent', 't5-base', 12)
defaultdict(<class 'list'>, {'Layer': [1, 2, 3, 4, 5, 6], 'Discourse Connective': [0.432, 0.4663333333333333, 0.48166666666666663, 0.5046666666666667, 0.511, 0.5226666666666667]})
   Layer
0      1
1      2
2      3
3      4
4      5
5      6
   Layer
0      1
1      2
2      3
3      4
4      5
5      6
